In [2]:
import numpy as np # NumPy is the fundamental package for scientific computing

import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

import matplotlib.pyplot as plt # Matplotlib is a python 2D plotting library
%matplotlib inline 
# A magic command that tells matplotlib to render figures as static images in the Notebook.

import seaborn as sns # Seaborn is a visualization library based on matplotlib (attractive statistical graphics).
sns.set_style('whitegrid') # One of the five seaborn themes
import warnings
warnings.filterwarnings('ignore') # To ignore some of seaborn warning msg

from scipy import stats

from sklearn import linear_model # Scikit learn library that implements generalized linear models
from sklearn import neighbors # provides functionality for unsupervised and supervised neighbors-based learning methods
from sklearn.metrics import mean_squared_error # Mean squared error regression loss
from sklearn import preprocessing # provides functions and classes to change raw feature vectors

from math import log

In [18]:
WDS = [
    # 高德
    ('b33e77f31fe04969803d8c93eee90701','41204679123','导航下载', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679138','语音导航', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679141','手机导航', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','52066904374','导航地图下载', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679126','手机导航下载', False,2.5),
    ('b33e77f31fe04969803d8c93eee90701','52257602639','手机导航地图',False,2.5),    
    ('b33e77f31fe04969803d8c93eee90701','41204679597','离线导航',False,2.5), 
    ('b33e77f31fe04969803d8c93eee90701','52257602606','手机导航下载',False,2.5),
    # 触宝
    ('99f34fe1ac3c472ca60f67296f2dabed','51851122225','有信电话免费下载', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683066718','微会下载打电话免费', True,2.0),
    #('99f34fe1ac3c472ca60f67296f2dabed','52549511179','移动无限流量', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683066700','微会', True,2.0),
    #('99f34fe1ac3c472ca60f67296f2dabed','52549510906','移动无限流量卡', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51662818937','网络电话', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683068212','免费电话软件', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683068299','免费电话', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683067966','网络电话永久免费打', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51662819099','网络电话软件', False,2.0),
]

def simple_linear_model(train, test, input_feature):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = [input_feature]), train.as_matrix(columns = ['profit'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['profit']), 
                              regr.predict(test.as_matrix(columns = [input_feature])))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_[0][0]

# A function that take multiple features as input and return the RMSE (of the test data), and the  intercept and coefficients
def multiple_regression_model(train, test, input_features):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = input_features), train.as_matrix(columns = ['profit'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['profit']), 
                              regr.predict(test.as_matrix(columns = input_features)))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_ 

# A function that take multiple features as input and return the RMSE (of the train and validation data)
def RMSE(train, validation, features, new_input):
    features_list = list(features)
    features_list.append(new_input)
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = features_list), train.as_matrix(columns = ['price'])) # Train the model
    RMSE_train = mean_squared_error(train.as_matrix(columns = ['price']), 
                              regr.predict(train.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    RMSE_validation = mean_squared_error(validation.as_matrix(columns = ['price']), 
                              regr.predict(validation.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    return RMSE_train, RMSE_validation 

for appid,kid,kname,rtb,revenue in WDS:
    if not rtb:
        continue
    print(kname,kid)
    data = pd.read_csv("%s.csv" % kid) # load the data into a pandas dataframe


    data.drop([
                'time_created',
                'chgange_id',
                'keyword',
                'keywordid',
                #'quality',
                #'time_hour',
                #'price',
                #'rank1rate',
                #'baidu_impression',
                'btn_impression',
                'baidu_click',
                'btn_click',
                'baidu_cost',
                'btn_cost',
                'baidu_cpc',
                'btn_cpc',
                'baidu_impression_increase_rate',
                'btn_impression_increase_rate',
                'baidu_click_increase_rate',
                'btn_click_increase_rate',
                'baidu_cost_increase_rate',
                'btn_cost_increase_rate',
                'h5_dl',
                'dl_total',
                'cost_per_dl',
                'hour_baidu_click',
                'hour_btn_click',
                #'profit'
              ], 
              axis = 1, inplace = True)
    #data.drop(['changeid', 'time','hourclick'], axis = 1, inplace = True)
    #data.head(2) # Show the first 2 lines
    #print(data.head(2))
    
    if False:
        dummies_zipcodes = pd.get_dummies(data['keywordid'], drop_first=False)
        dummies_zipcodes.reset_index(inplace=True)
        dummies_zipcodes = dummies_zipcodes.add_prefix("{}#".format('keywordid'))
        dummies_zipcodes = dummies_zipcodes[['keywordid#41204679123',
                                         'keywordid#41204679138',
                                         'keywordid#41204679141',
                                         'keywordid#51851122225',
                                         'keywordid#51683066718',
                                         'keywordid#52549511179',
                                         'keywordid#51683066700',
                                         'keywordid#52549510906',
                                        ]]
        data.drop('keywordid', axis=1, inplace=True)
        data = data.join(dummies_zipcodes)

    data.dtypes

    from sklearn.cross_validation import train_test_split
    train_data, test_data = train_test_split(data, train_size = 0.8, random_state = 10)
    # A function that take one input of the dataset and return the RMSE (of the test data), and the intercept and coefficient

    input_list = data.columns.values.tolist() # list of column name
    input_list.remove('profit')
    simple_linear_result = pd.DataFrame(columns = ['feature', 'RMSE', 'intercept', 'coefficient'])

    # 查看所有参数的线性模型参数和RMSE
    if  False:
        # loop that calculate the RMSE of the test data for each input 
        for p in input_list:
            RMSE1, w1, w0 = simple_linear_model(train_data, test_data, p)
            simple_linear_result = simple_linear_result.append({'feature':p, 'RMSE':RMSE1, 'intercept':w0, 'coefficient': w1}
                                                       ,ignore_index=True)
        
        aa = simple_linear_result.sort_values('RMSE').head(15) # display the 10 best estimators
        print(aa)

    train_data['time_hour_2'] = train_data['time_hour'].apply(lambda x: x**2)
    test_data['time_hour_2'] = test_data['time_hour'].apply(lambda x: x**2)

    train_data['price_2'] = train_data['price'].apply(lambda x: x**2)
    test_data['price_2'] = test_data['price'].apply(lambda x: x**2) 

    train_data['rank1_2'] = train_data['rank1rate'].apply(lambda x: x**2)
    test_data['rank1_2'] = test_data['rank1rate'].apply(lambda x: x**2)
    
    train_data['quality_2'] = train_data['quality'].apply(lambda x: x**2)
    test_data['quality_2'] = test_data['quality'].apply(lambda x: x**2)
    
    train_data['baidu_impression_2'] = train_data['baidu_impression'].apply(lambda x: x**2)
    test_data['baidu_impression_2'] = test_data['baidu_impression'].apply(lambda x: x**2)

    print('kid-%s' % kid)

    
    #print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['price','rank1rate']))
    #print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['price','hour']))
    #print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data,test_data, ['hour','rank1rate']))
    
    train_data_2, validation_data = train_test_split(train_data, train_size = 0.75, random_state = 50)
    input_list = train_data_2.columns.values.tolist() # list of column name
    input_list.remove('profit')

    # list of features included in the regression model and the calculated train and validation errors (RMSE)
    regression_greedy_algorithm = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])  
    i = 0
    temp_list = []

    # a while loop going through all the features in the dataframe
    while i < len(train_data_2.columns)-1: 
        # a temporary dataframe to select the best feature at each iteration
        temp = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])
    
        # a for loop to test all the remaining features
        for p in input_list:
            RMSE_train, RMSE_validation = RMSE(train_data_2, validation_data, temp_list, p)
            temp = temp.append({'feature':p, 'train_error':RMSE_train, 'validation_error':RMSE_validation}, ignore_index=True)
        
        temp = temp.sort_values('train_error') # select the best feature using train error
        best = temp.iloc[0,0]
        temp_list.append(best)
        regression_greedy_algorithm = regression_greedy_algorithm.append({'feature': best, 
                                                  'train_error': temp.iloc[0,1], 'validation_error': temp.iloc[0,2]}, 
                                                 ignore_index=True) # add the feature to the dataframe
        input_list.remove(best) # remove the best feature from the list of available features
        i += 1
    print(regression_greedy_algorithm)
    
    print('\n')
    greedy_algo_features_list = regression_greedy_algorithm['feature'].tolist()[:12] # select the first 30 features
    test_error, _, _ = multiple_regression_model(train_data_2, test_data, greedy_algo_features_list)
    print ('test error (RMSE) is: %s' %test_error)

    print('------------------\n')

导航下载 41204679123
kid-41204679123
              feature   train_error  validation_error
0               price  0.000000e+00      0.000000e+00
1             quality  0.000000e+00      0.000000e+00
2         time_hour_2  0.000000e+00      0.000000e+00
3           quality_2  0.000000e+00      0.000000e+00
4           time_hour  1.170278e-16      1.110223e-16
5    baidu_impression  1.812987e-16      2.719480e-16
6  baidu_impression_2  3.431928e-16      6.798700e-16
7             rank1_2  1.539705e-15      3.454649e-15
8           rank1rate  4.853479e-16      6.707438e-16
9             price_2  2.386656e-12      7.482424e-11


test error (RMSE) is: 73.7549551658
------------------

语音导航 41204679138
kid-41204679138
              feature   train_error  validation_error
0               price  5.233642e-17      0.000000e+00
1           time_hour  3.700743e-17      0.000000e+00
2  baidu_impression_2  0.000000e+00      2.775558e-17
3             quality  5.233642e-17      0.000000e+00
4    baidu_i

In [2]:

greedy_algo_features_list = regression_greedy_algorithm['feature'].tolist()[:12] # select the first 30 features
test_error, _, _ = multiple_regression_model(train_data_2, test_data, greedy_algo_features_list)
print ('test error (RMSE) is: %s' %test_error)

NameError: name 'regression_greedy_algorithm' is not defined

In [77]:

input_feature = train_data.columns.values.tolist() # list of column name
input_feature.remove('cpc')

for i in [1,10]:
    ridge = linear_model.Ridge(alpha = i, normalize = True) # initialize the model
    ridge.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              ridge.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

test error (RMSE) is: 0.383967842026
test error (RMSE) is: 0.38059134014


In [78]:
ridgeCV = linear_model.RidgeCV(alphas = np.linspace(1.0e-10,1,num = 100), normalize = True, store_cv_values = True) # initialize the model
ridgeCV.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
print ('best alpha is: %s' %ridgeCV.alpha_) # get the best alpha
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              ridgeCV.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

best alpha is: 0.0303030304
test error (RMSE) is: 0.387848881773


In [79]:
for i in [0.01,0.1,1,250,500,1000]:
    lasso = linear_model.Lasso(alpha = i, normalize = True) # initialize the model
    lasso.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['cpc'])) # fit the train data
    print (lasso.sparse_coef_.getnnz) # number of non zero weights
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['cpc']), 
                              lasso.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.379868689916
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 0.380408408688
<bound method _cs_matrix.getnnz of <1x7 sparse matrix of type '<class 'numpy.float64'

In [26]:
lassoCV = linear_model.LassoCV(normalize = True) # initialize the model (alphas are set automatically)
lassoCV.fit(train_data.as_matrix(columns = input_feature), np.ravel(train_data.as_matrix(columns = ['price']))) # fit the train data
print ('best alpha is: %s' %lassoCV.alpha_) # get the best alpha
print ('number of non zero weigths is: %s' %np.count_nonzero(lassoCV.coef_)) # number of non zero weights
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              lassoCV.predict(test_data.as_matrix(columns = [input_feature])))**0.5) # predict price and test error

best alpha is: 12.1151133713
number of non zero weigths is: 39
test error (RMSE) is: 171760.045248


In [27]:
# normalize the data
train_X = train_data.as_matrix(columns = input_feature)
scaler = preprocessing.StandardScaler().fit(train_X)
train_X_scaled = scaler.transform(train_X)
test_X = test_data.as_matrix(columns = [input_feature])
test_X_scaled = scaler.transform(test_X)

knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance') # initialize the model
knn.fit(train_X_scaled, train_data.as_matrix(columns = ['price'])) # fit the train data
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              knn.predict(test_X_scaled))**0.5) # predict price and test error

test error (RMSE) is: 183023.744768
